In [6]:
import pandas as pd
from sklearn.linear_model import LogisticRegression, SGDClassifier, SGDOneClassSVM
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score
from sklearn.metrics import * 
from imblearn.over_sampling import SMOTE

import warnings    

In [7]:
import warnings
warnings.filterwarnings('ignore')

!pip install imblearn

In [8]:
#csv로드
#'drugs_related_stop' 열 예측
trafficData = pd.read_csv('traffic_violaions.csv')
trafficData=trafficData.dropna()
trafficData['drugs_related_stop'] = trafficData['drugs_related_stop'].replace({True: 1, False: 0})
X = trafficData[['driver_age', 'driver_gender', 'stop_time', 'is_arrested', 'violation']]
y = trafficData['drugs_related_stop']
new_x = trafficData[['driver_age', 'driver_gender', 'stop_time', 'is_arrested', 'violation']]
new_x.driver_gender = new_x.driver_gender.apply(lambda x: 0 if x == 'M' else 1)
new_x.stop_time = new_x.stop_time.apply(lambda x: int(x.split(':')[0]))
new_x.is_arrested = new_x.is_arrested.astype(int)
trafficData.violation.unique()
xx = pd.concat([new_x, pd.get_dummies(new_x.violation)], axis = 1)
xx = xx.drop('violation', axis = 1)

,stop_date,stop_time,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,stop_outcome,is_arrested,stop_duration,drugs_related_stop
0,1/2/2005,1:55,M,1985.0,20.0,White,Speeding,Speeding,Citation,False,0-15 Min,0
1,1/18/2005,8:15,M,1965.0,40.0,White,Speeding,Speeding,Citation,False,0-15 Min,0
2,1/23/2005,23:15,M,1972.0,33.0,White,Speeding,Speeding,Citation,False,0-15 Min,0
3,2/20/2005,17:15,M,1986.0,19.0,White,Call for Service,Other,Arrest Driver,True,16-30 Min,0
4,3/14/2005,10:00,F,1984.0,21.0,White,Speeding,Speeding,Citation,False,0-15 Min,0
...,...,...,...,...,...,...,...,...,...,...,...,...
52956,10/4/2011,22:22,M,1979.0,32.0,White,Speeding,Speeding,Arrest Driver,True,0-15 Min,0
52958,10/4/2011,22:36,F,1967.0,44.0,Hispanic,Speeding,Speeding,Citation,False,0-15 Min,0
52959,10/4/2011,23:25,F,1991.0,20.0,Black,Speeding,Speeding,Citation,False,0-15 Min,0
52962,10/5/2011,6:43,M,1986.0,25.0,White,Speeding,Speeding,Citation,False,0-15 Min,0


In [9]:
xx

,driver_age,driver_gender,stop_time,is_arrested,Equipment,Moving violation,Other,Registration/plates,Speeding
0,20.0,0,1,0,0,0,0,0,1
1,40.0,0,8,0,0,0,0,0,1
2,33.0,0,23,0,0,0,0,0,1
3,19.0,0,17,1,0,0,1,0,0
4,21.0,1,10,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...
52956,32.0,0,22,1,0,0,0,0,1
52958,44.0,1,22,0,0,0,0,0,1
52959,20.0,1,23,0,0,0,0,0,1
52962,25.0,0,6,0,0,0,0,0,1


In [61]:
new_x

,driver_age,driver_gender,stop_time,is_arrested,violation
0,20.0,0,1,0,Speeding
1,40.0,0,8,0,Speeding
2,33.0,0,23,0,Speeding
3,19.0,0,17,1,Other
4,21.0,1,10,0,Speeding
...,...,...,...,...,...
52956,32.0,0,22,1,Speeding
52958,44.0,1,22,0,Speeding
52959,20.0,1,23,0,Speeding
52962,25.0,0,6,0,Speeding


In [15]:
y.value_counts()

0    48944
1      396
Name: drugs_related_stop, dtype: int64

In [23]:

X_train, X_test, y_train, y_test = train_test_split(xx, y, test_size=0.3,  random_state=11)
X_train, x_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=11)

X_train.shape, x_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape

((24176, 9), (10362, 9), (14802, 9), (24176,), (10362,), (14802,))

In [24]:
pd.DataFrame(y_train).value_counts()
 

drugs_related_stop
0                     23976
1                       200
dtype: int64

In [25]:
smote = SMOTE(random_state= 10)
x_train_over, y_train_over = smote.fit_resample(X_train, y_train)
x_train_over.shape, y_train_over.shape
 


((47952, 9), (47952,))

In [26]:
pd.DataFrame(y_train_over).value_counts()

drugs_related_stop
0                     23976
1                     23976
dtype: int64

In [29]:

lr = LogisticRegression(max_iter = 100)
lr.fit(x_train_over, y_train_over)
y_lr_pred = lr.predict(X_test)
f1_score(y_test, y_lr_pred), accuracy_score(y_test, y_lr_pred)

(0.06380208333333333, 0.9028509660856641)

In [30]:
pd.DataFrame(y_lr_pred).value_counts()

0    13386
1     1416
dtype: int64

In [31]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(x_train_over, y_train_over)
y_pred = lr.predict(X_test)
f1_score(y_test, y_pred), accuracy_score(y_test, y_pred)

(0.06380208333333333, 0.9028509660856641)

In [36]:
import joblib
joblib.dump(model, 'saved_model.pkl')


['saved_model.pkl']

In [119]:
import pandas as pd
import joblib

#모델 로드
model = joblib.load('C:\machine_learning\cgi-bin\saved_model.pkl')

#데이터 인풋
input_data = pd.DataFrame({
    'driver_age': [26],
    'driver_gender': ['M'],
    'stop_time': [10],
    'is_arrested': [False],
    'Equipment' :[0],
    'Moving violation' :[0],
    'Other' :[0],
    'Registration/plates':[0],
    'Speeding': [1]
})
input_data
input_data.driver_gender = input_data.driver_gender.apply(lambda x: 0 if x == 1 else 1)
input_data.stop_time = input_data.stop_time.astype(int)
input_data.is_arrested = input_data.is_arrested.astype(int)
input_data
xx=input_data


prediction = model.predict(xx)

if prediction[0] == 0:
    print("이 교통 위반자는 마약과 관련이 없습니다.")
else:
    print("마약관련 교통위반자입니다.")

이 교통 위반자는 마약과 관련이 없습니다.
